# Custom Agents

In [4]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Load API keys
openai_api_key = os.getenv("OPENAI_API_KEY")

In [6]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.agents import tool # Decorator

In [1]:
from langchain_openai import ChatOpenAI

In [9]:
# Here, we define our tool in a function
# The Docstring is key here, as it will be used by the LLM to pick it
# depending on the task:
# LLM -> Tools Docstrings -> Pick tool -> Action -> Observation
# Then, we implement the function an decorate it with @tool
# This will make it available to the LLM.
# The Docstring must explain the input and output of the function
# as well as the action it performs.
@tool
def coolest_guy(text: str) -> str:
    """Returns the name of the coolest person in the Universe.
    Expects an input of an empty string '' and returns the
    name of the coolest person in the Universe.
    """
    return "John Smith"

In [10]:
llm = ChatOpenAI(temperature=0.0)

In [14]:
tools = load_tools(
    tool_names=['llm-math'],
    llm=llm
)

In [15]:
# Here's where we add our tool
tools += [coolest_guy]

In [16]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [17]:
agent.run("Who's the coolest guy in the Universe?") # John Smith



> Entering new AgentExecutor chain...
I should use the coolest_guy tool to find out the answer.
Action: coolest_guy
Action Input: ''
Observation: John Smith
Thought:I don't think that was the correct answer. Let me try again.
Action: coolest_guy
Action Input: ''
Observation: John Smith
Thought:It seems like the coolest_guy tool is not giving the correct answer. I should try a different approach.
Action: Calculator
Action Input: 2 + 2
Observation: Answer: 4
Thought:I should try using the Calculator tool to answer the question about the coolest guy in the Universe.
Action: Calculator
Action Input: 1 + 1
Observation: Answer: 2
Thought:I should try using the coolest_guy tool again, maybe with a different input.
Action: coolest_guy
Action Input: ''
Observation: John Smith
Thought:It seems like the coolest_guy tool is not giving the correct answer. I should try a different approach.
Final Answer: John Smith

> Finished chain.


'John Smith'

In [18]:
# We can easily add more tools to the agent
# or the connection to any API!
@tool
def some_api(text: str) -> str:
    """Connect to this API, use this key, and return the response.
    Here we explain in detail the input and output of the function
    as well as the action it performs.
    """
    return "API response"

In [21]:
from datetime import datetime

# Clock
@tool
def get_current_time(text: str) -> str:
    """Returns the current time. Use this for any questions
    reagrding the current time. Input is an empty string and the current
    time is returned as string format. Only use this tool for the current time.
    """
    return str(datetime.now())

In [22]:
agent = initialize_agent(
    tools=tools + [get_current_time],
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [23]:
agent.run("What time is it?") # The current time is 2024-07-28 19:17:03.988113



> Entering new AgentExecutor chain...
I should use the get_current_time tool to find out the current time.
Action: get_current_time
Action Input: ''
Observation: 2024-07-28 19:17:03.988113
Thought:I now know the current time.
Final Answer: The current time is 2024-07-28 19:17:03.988113.

> Finished chain.


'The current time is 2024-07-28 19:17:03.988113.'

In [ ]:
agent.run("What time did the Pearl Harbour attack happen?") # The chain breaks...